Circle–Circle Intersection (Solver Geometry)

Given:

r1 = l2

r2 = l3

1) Center distance
d = |C − A|

2) Distance to chord center
a = (r1^2 − r2^2 + d^2) / (2 d)

3) Half-chord length
h = sqrt(r1^2 − a^2)

4) Chord center point
P = A + a (C − A) / d

5) Intersection points
Let n_hat be a unit vector perpendicular to (C − A).

B = P ± h n_hat

The solver selects + or − based on the desired four-bar assembly.


In [16]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Arc
from IPython.display import HTML

# -----------------------------
# Link lengths (EDIT THESE)
# -----------------------------
l_1 = 3.0
l_2 = 5.0
l_3 = 4.0
l_4 = 4.0
l_5 = 4.0   # perpendicular, ORIGIN at C

# -----------------------------
# Theta1 range (input crank)
# -----------------------------
theta1 = np.linspace(0, np.pi/2, 400)

# -----------------------------
# Ground points
# -----------------------------
O = np.array([0.0, 0.0])
C = np.array([0.0, l_4])

# Fixed target point
E = np.array([C[0] - 6.0, C[1]])

# -----------------------------
# Storage for joints
# -----------------------------
A = np.zeros((len(theta1), 2))
B = np.zeros((len(theta1), 2))
D = np.zeros((len(theta1), 2))

# -----------------------------
# Geometry solver
# -----------------------------
for i, t1 in enumerate(theta1):

    A[i] = O + np.array([l_1*np.cos(t1), l_1*np.sin(t1)])

    d = np.linalg.norm(C - A[i])

    if d > l_2 + l_3 or d < abs(l_2 - l_3):
        B[i] = np.nan
        D[i] = np.nan
        continue

    a = (l_2**2 - l_3**2 + d**2) / (2*d)
    h = np.sqrt(l_2**2 - a**2)
    P = A[i] + a*(C - A[i]) / d

    B[i,0] = P[0] + h*(C[1] - A[i,1]) / d
    B[i,1] = P[1] - h*(C[0] - A[i,0]) / d

    v3 = C - B[i]
    v3_hat = v3 / np.linalg.norm(v3)

    # Perpendicular link 5
    n_hat = np.array([v3_hat[1], -v3_hat[0]])
    D[i] = C + l_5 * n_hat

# -----------------------------
# Remove invalid frames
# -----------------------------
valid = ~np.isnan(B[:,0])
theta1 = theta1[valid]
A = A[valid]
B = B[valid]
D = D[valid]

# -----------------------------
# Angle helpers
# -----------------------------
def angle_deg(p1, p2):
    v = p2 - p1
    return np.degrees(np.arctan2(v[1], v[0]))

def small_angle_deg(th):
    th = (th + 180) % 360 - 180
    if th > 90:
        th -= 180
    elif th < -90:
        th += 180
    return th



In [ ]:
# -----------------------------
# Plot limits
# -----------------------------
all_x = np.hstack([O[0], C[0], E[0], A[:,0], B[:,0], D[:,0]])
all_y = np.hstack([O[1], C[1], E[1], A[:,1], B[:,1], D[:,1]])
pad = 0.5
xmin, xmax = all_x.min()-pad, all_x.max()+pad
ymin, ymax = all_y.min()-pad, all_y.max()+pad

# -----------------------------
# Animation setup
# -----------------------------
fig, ax = plt.subplots(figsize=(8,6))
ax.set_aspect('equal')
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.grid(True)

link,  = ax.plot([], [], lw=3)
link5, = ax.plot([], [], lw=3, color='green')
spring, = ax.plot([], [], lw=3, color='purple')
traceB, = ax.plot([], [], lw=1, alpha=0.6)
traceD, = ax.plot([], [], lw=1, linestyle='--')
joints, = ax.plot([], [], 'o')
target, = ax.plot(E[0], E[1], 's', color='purple')

traceBx, traceBy = [], []
traceDx, traceDy = [], []

angle_text = ax.text(0.02, 0.98, "", transform=ax.transAxes,
                     va='top', ha='left', fontsize=10, family="monospace")

# -----------------------------
# Angle arcs (visual)
# -----------------------------
arc1 = Arc(O, 1.0, 1.0, theta1=0, theta2=0, color='black', lw=2)
arc2 = Arc(A[0], 1.0, 1.0, theta1=0, theta2=0, color='black', lw=2)
arc3 = Arc(C, 1.0, 1.0, theta1=0, theta2=0, color='black', lw=2)
arc4 = Arc(C, 1.2, 1.2, theta1=0, theta2=0, color='green', lw=2)
arc5 = Arc(E, 1.0, 1.0, theta1=0, theta2=0, color='purple', lw=2)

for arc in [arc1, arc2, arc3, arc4, arc5]:
    ax.add_patch(arc)

# -----------------------------
# Init
# -----------------------------
def init():
    link.set_data([], [])
    link5.set_data([], [])
    spring.set_data([], [])
    traceB.set_data([], [])
    traceD.set_data([], [])
    joints.set_data([], [])
    angle_text.set_text("")
    return link, link5, spring, traceB, traceD, joints, arc1, arc2, arc3, arc4, arc5, angle_text

# -----------------------------
# Update
# -----------------------------
def update(i):
    # Main 4-bar
    x = [O[0], A[i,0], B[i,0], C[0], O[0]]
    y = [O[1], A[i,1], B[i,1], C[1], O[1]]
    link.set_data(x, y)

    # Link 5
    link5.set_data([C[0], D[i,0]], [C[1], D[i,1]])

    # Spring
    spring.set_data([D[i,0], E[0]], [D[i,1], E[1]])

    # Traces
    traceBx.append(B[i,0])
    traceBy.append(B[i,1])
    traceB.set_data(traceBx, traceBy)

    traceDx.append(D[i,0])
    traceDy.append(D[i,1])
    traceD.set_data(traceDx, traceDy)

    # Joints
    joint_x = [A[i,0], B[i,0], C[0], D[i,0]]
    joint_y = [A[i,1], B[i,1], C[1], D[i,1]]
    joints.set_data(joint_x, joint_y)

    # -----------------------------
    # Angles (numerical)
    # -----------------------------
    th1 = angle_deg(O, A[i])
    th2 = angle_deg(A[i], B[i])
    th3 = angle_deg(C, B[i])
    th4 = small_angle_deg(-angle_deg(C, D[i]))   # CW numeric (unchanged)
    th5 = angle_deg(E, D[i])

    angle_text.set_text(
        f"θ1 (O→A): {th1:7.2f}°\n"
        f"θ2 (A→B): {th2:7.2f}°\n"
        f"θ3 (C→B): {th3:7.2f}°\n"
        f"θ4 (C→D, CW): {th4:7.2f}°\n"
        f"θ5 (E→D): {th5:7.2f}°"
    )

    # -----------------------------
    # UPDATE VISUAL ARCS
    # -----------------------------
    arc1.center = O
    arc1.theta1 = 0
    arc1.theta2 = th1

    arc2.center = A[i]
    arc2.theta1 = 0
    arc2.theta2 = th2

    arc3.center = C
    arc3.theta1 = 0
    arc3.theta2 = th3

    # θ5: start from link, end at 180°, CCW
    link5_ccw = angle_deg(C, D[i])
    arc4.center = C
    arc4.theta1 = link5_ccw
    arc4.theta2 = 180

    arc5.center = E
    arc5.theta1 = 0
    arc5.theta2 = th5

    # -----------------------------
    # Labels for points
    # -----------------------------
    # Remove previous labels except angle_text
    for txt in ax.texts[1:]:
        txt.remove()

    offset = 0.15
    ax.text(O[0]-offset, O[1]-2*offset, 'O', color='black')
    ax.text(A[i,0]+offset, A[i,1]+offset, 'A', color='black')
    ax.text(B[i,0]+offset, B[i,1]+offset, 'B', color='black')
    ax.text(C[0]-offset, C[1]+offset, 'C', color='black')
    ax.text(D[i,0]+offset, D[i,1]+offset, 'D', color='green')
    ax.text(E[0]-offset, E[1]+offset, 'E', color='purple')

    return link, link5, spring, traceB, traceD, joints, arc1, arc2, arc3, arc4, arc5, angle_text

# -----------------------------
# Create animation
# -----------------------------
ani = FuncAnimation(
    fig,
    update,
    frames=len(A),
    init_func=init,
    interval=30,
    blit=False
)

# -----------------------------
# DISPLAY INLINE
# -----------------------------
HTML(ani.to_jshtml())
